## Import libraries

In [2]:
# torch imports
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
import torch.functional as F

# l5kit imports
import l5kit
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory

# common imports
import os
import random
import time
from typing import Dict
from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from prettytable import PrettyTable
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

In [3]:
l5kit.__version__

'1.1.0'

## Helper functions

### Loss function

In [21]:
# --- Function utils ---
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py
from torch import Tensor


def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

### Random seed generation function

In [4]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed(42)

### Resnet forward function

In [31]:
def resnet_forward(backbone, x):    
    #with torch.set_grad_enabled(False):
    with torch.no_grad():
        x = backbone.conv1(x)
        x = backbone.bn1(x)
        x = backbone.relu(x)
        x = backbone.maxpool(x)

        x = backbone.layer1(x)
        x = backbone.layer2(x)
        x = backbone.layer3(x)
        x = backbone.layer4(x)

        x = backbone.avgpool(x)
        x = torch.flatten(x, 1)
    return x

### Number of trainable parameters in model

In [25]:
def find_no_of_trainable_params(model):
    total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(total_trainable_params)

## Base LSTM Structure

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

## Configs

In [5]:
# --- Lyft configs ---
cfg = {
    'format_version': 4,
    'data_path': "lyft-motion-prediction-autonomous-vehicles/",
    'model_params': {
        'model_architecture': 'resnet34',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
        'model_name': "LSTM_baseline_r34",
        'lr': 1e-3,
        'weight_path': "lyft-pretrained-model-hv/model_multi_update_lyft_public.pth",
        'train': True,
        'predict': False
    },

    'raster_params': {
        'raster_size': [224, 224],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },

    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 4
    },
    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 4
    },

    'sample_data_loader': {
        'key': 'scenes/sample.zarr',
        'batch_size': 16,
        'shuffle': True,
        'num_workers': 4
    },

    'train_params': {
        'max_num_steps': 5000,
        'checkpoint_every_n_steps': 500,
    }
}

## Load the training data

In [23]:
# set env variable for data
DIR_INPUT = cfg["data_path"]
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)
rasterizer = build_rasterizer(cfg, dm)

In [9]:
# ===== INIT TRAIN DATASET============================================================
train_cfg = cfg["train_data_loader"]
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)

starting pool...


16265/16265: : 16265it [18:42, 14.49it/s]


collecting results..
start report for lyft-motion-prediction-autonomous-vehicles/scenes/train.zarr
{   'reject_th_AV_distance': 40138602,
    'reject_th_agent_filter_probability_threshold': 214185074,
    'reject_th_extent': 13797837,
    'reject_th_yaw': 824397,
    'th_agent_filter_probability_threshold': 0.5,
    'th_distance_av': 50,
    'th_extent_ratio': 1.1,
    'th_yaw_degree': 30,
    'total_agent_frames': 320124624,
    'total_reject': 268945910}


computing past/future table: 100%|██████████| 4/4 [00:21<00:00,  5.41s/it]


+-------------+-----------+----------+----------+---------+
| past/future |     0     |    10    |    30    |    50   |
+-------------+-----------+----------+----------+---------+
|      0      | 320124624 | 23355429 | 13208858 | 8725660 |
|      10     |  23355429 | 17003687 | 10609787 | 7335740 |
|      30     |  13208858 | 10609787 | 7335740  | 5400884 |
|      50     |  8725660  | 7335740  | 5400884  | 4073267 |
+-------------+-----------+----------+----------+---------+
end report for lyft-motion-prediction-autonomous-vehicles/scenes/train.zarr


In [11]:
print('Length of Train dataset is ' ,len(train_dataset))
print("==================================TRAIN DATA==================================")
print(train_dataset)

Length of Train dataset is  22496709
==================================TRAIN DATA==================================
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16265    |  4039527   | 320124624  |    38735988   |      112.19     |        248.36        |        79.25         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


In [12]:
sampled_indices = np.random.choice(len(train_dataset), size = len(train_dataset), replace = False)
print(sampled_indices[0:10])
Datasampler = SubsetRandomSampler(sampled_indices)

[ 4544818 11783688  2821987 10532781  1993967 15711361  7726097   547516
 17556178  7275034]


In [13]:
train_dataloader = DataLoader(train_dataset, sampler=Datasampler, batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])

## CUDA device && encoding model

In [14]:
# ==== INIT MODEL=================
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device {device}')

device cuda:0


In [17]:
encoding_model = resnet34(pretrained=True)
encoding_model.to(device);

# Freeze parameters so we don't backprop through them
for param in encoding_model.parameters():
    param.requires_grad = False

Total_trainable_params = find_no_of_trainable_params(encoding_model)
print(f'There are {Total_trainable_params} trainable parameters in the model')

# set to evaluation mode
encoding_model.eval();

In [33]:
train_itr = iter(train_dataloader)
data = next(train_itr)

In [34]:
inputs = data["image"].to(device)
target_availabilities = data["target_availabilities"].to(device)
targets = data["target_positions"].to(device)

In [36]:
%%time

# agent frames
agent_data = inputs[:, 0:11, :,:]
#print(agent_data.shape)

# ego vehicle frames
ego_data = inputs[:, 11:-3, :,:]
#print(ego_data.shape)

# combined ego and agent frames, duplicating across 3 channels
vehicle_data = torch.repeat_interleave(ego_data + agent_data, 3, dim=1)

# pretrained model requires (batch_size, 3, 224, 224), hence reshaping
vehicle_data = vehicle_data.view(-1, 3, 224, 224)
print(vehicle_data.shape)

AttributeError: module 'torch' has no attribute 'repeat_interleave'